# Archery Capstone
This capstone project was created in support of the CodeYou program to showcase the materials covered in the Data Analysis pathway. 
The code blocks below will walk you through the following:<br>
1. Accessing three layers created within ArcGIS Online to collect archery related stats from 3D target events<br>
2. Pulling these layers down to data frames to work with separately from the authoritative data within ArcGIS Online<br>
3. Cleaning and joining the data so it can be used for futher analysis and graphic creation<br>
4. Creating graphics and shot summaries for the archer to reference and analyze for trends

## Data Collection
Data was created and collected using esri's ArcGIS Online platform in support of this project. Three distinct layers will be referenced for this project:<br>
1.  A lookup table was created containing the various 3D target animals along with information on the color, size, name, and target pattern.<br> 
2.  A Survey123 was created to collect shoot conditions for each tournament.  Data was collected to capture the shoot name, shoot location (indoor or outdoor), shoes (minimal, hiking boots, sandals, gym shoes), temperature, weather conditions present (sunny, cloudy, humid, windy, foggy, light rain, heavy rain, stormy), and personal conditions (well rested, tired, sick, sore, hungry, full).<br>
3.  A Survey123 was created to collect several details for each shot on the individual target and appearance (animal, range, position, elevation, lighting), shot execution (confidence, release details, execution details), and placement (score, aim placement, actual arrow impact). 

### Import Modules

This section imports the various modules required for the code to execute. 

In [66]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd
import pandasql as psql
import requests
from IPython.display import display
print("Modules Imported")

Modules Imported


### Define Variables

This section defines any variables which may be referenced in subsequent code blocks below.

### Download AGOL layers to data frames

This section connects to ArcGIS Online and downloads two of the three datasets to dataframes for cleaning and analysis. The final dataset is then converted to a dataframe from a CSV. 

In [67]:
# Connect to ArcGIS Online
# gis = GIS("https://sso.maps.arcgis.com/")
gis = GIS()

# Define ItemIDs for AGOL layers
Shot = "f0314c505e83428d8ce6894a8d73eb09"
Shoot = "5fa46f5f47e841b1b2454271cd346d8f"

# Search for the layers by title
layer3_Shoot = gis.content.get(Shoot)
print (layer3_Shoot.type)
layer2_Shot = gis.content.get(Shot)
print (layer2_Shot.type)


# Access the first layer in each item
feature_layer2_Shot = layer2_Shot.layers[0]
feature_layer3_Shoot = layer3_Shoot.layers[0]
print (feature_layer2_Shot)
print (feature_layer3_Shoot)

# Query the layers to get all features
features2 = feature_layer2_Shot.query()
features3 = feature_layer3_Shoot.query()

# Convert features to pandas DataFrames
df2_Shot = features2.sdf
df3_Shoot = features3.sdf

# Read the AnimalLookup CSV to a dataframe
animalLookup_path = r"C:\Users\emily.bartee\Documents\Projects\CapstoneArchery\CapstoneArchery\Data\AnimalLookup.csv"
df1_Animal = pd.read_csv(animalLookup_path)


Feature Service
Feature Service
<FeatureLayer url:"https://services6.arcgis.com/MQNEtGZdkDmSDKfq/arcgis/rest/services/survey123_c0aeabe39091491e937f8938d064f6f8/FeatureServer/0">
<FeatureLayer url:"https://services6.arcgis.com/MQNEtGZdkDmSDKfq/arcgis/rest/services/survey123_4cefed1f94564642baf9db2aeb6e23a3/FeatureServer/0">


In [68]:
# Now that our data has been downloaded to dataframes, let's make sure we are able to see all resulting columns and rows. 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Explore the columns and data of each data frame
print(f"Animal Lookup Dataframe: {df1_Animal.columns}")
display(df1_Animal)
print(f"Shot Report Out Dataframe: {df2_Shot.columns}")
display(df2_Shot)
print(f"Shoot Details: {df3_Shoot.columns}")
display(df3_Shoot)

Animal Lookup Dataframe: Index(['OBJECTID', 'Target Name', 'Target Color', 'Target Size',
       'Target Pattern'],
      dtype='object')


,OBJECTID,Target Name,Target Color,Target Size,Target Pattern
0,1,Aoudad,Brown,Big,No
1,2,Bighorn Sheep,Brown,Big,No
2,3,Black Bear,Black,Medium,No
3,4,Black Buck,Black and White,Big,No
4,5,Black Panther,Black,Medium,No
5,6,Blesbok,Black,Big,No
6,7,Brown Bear,Brown,Medium,No
7,8,Chamois,Brown,Big,No
8,9,Coyote,Brown,Small,No
9,10,Grazing Doe,Brown,Big,No


Shot Report Out Dataframe: Index(['objectid', 'globalid', 'CreationDate', 'Creator', 'EditDate', 'Editor',
       'target_order', 'date_and_time', 'target', 'untitled_question_3_other',
       'range', 'position', 'elevation', 'lighting', 'shot_confidence',
       'shot_placement', 'aim_placement', 'shot_execution',
       'shot_execution_other', 'score', 'shooter_name', 'release_details',
       'scored_ring', 'adjustment_made', 'adjustment', 'SHAPE'],
      dtype='object')


,objectid,globalid,CreationDate,Creator,EditDate,Editor,target_order,date_and_time,target,untitled_question_3_other,range,position,elevation,lighting,shot_confidence,shot_placement,aim_placement,shot_execution,shot_execution_other,score,shooter_name,release_details,scored_ring,adjustment_made,adjustment,SHAPE
0,1,18762273-14e9-4bb4-836b-99d5a2fb6ff8,2024-06-28 13:06:53.326999,emily.bartee,2024-06-28 13:06:53.326999,emily.bartee,1.0,2024-06-28 12:42:00,Javelina,,37.6,Broadside,Level,Part Shadow,High,On_Aim_Point,Lower 12,Felt_Good,,12.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.85113242, ""y"": 37.27184107, ""spatial..."
1,2,2ed4aa14-1f33-4dd4-bb7a-fd11613915d2,2024-06-28 13:11:29.308999,emily.bartee,2024-06-28 13:11:29.308999,emily.bartee,2.0,2024-06-28 13:07:00,Grazing Doe,,30.4,Broadside,Level,Full Shadow,Low,"High,Left",Lower 12,Bad_Release,,10.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.85128708, ""y"": 37.27157439, ""spatial..."
2,3,30fe7f13-d439-4568-ba63-3b512976126c,2024-06-28 13:17:34.042000,emily.bartee,2024-06-28 13:17:34.042000,emily.bartee,3.0,2024-06-28 13:15:00,Brown Bear,,39.3,Broadside,Level,Full Shadow,Low,Left,Lower 12,"other,Felt_Good",Pull through,10.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.85100958, ""y"": 37.27168401, ""spatial..."
3,4,5c5f1906-e865-4bac-a20b-cf30e93321bb,2024-06-28 13:22:40.687000,emily.bartee,2024-06-28 13:22:40.687000,emily.bartee,4.0,2024-06-28 13:19:00,Grazing Doe,,31.3,Broadside,Level,Full Shadow,High,High,Lower 12,Push_Up,,10.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.85095463, ""y"": 37.27159442, ""spatial..."
4,5,0d1918ec-6ca2-42ff-bc48-fff9edf9e7d1,2024-06-28 13:29:50.051000,emily.bartee,2024-06-28 13:29:50.051000,emily.bartee,<NA>,2024-06-28 13:27:00,Black Panther,,41.0,Quartered_From,Level,Full Shadow,Low,On_Aim_Point,Center 10,Glare,,10.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.85093721, ""y"": 37.27152336, ""spatial..."
5,6,a2e02b93-93d5-41b8-92b2-64e074a08540,2024-06-28 14:12:14.900000,emily.bartee,2024-06-28 14:12:14.900000,emily.bartee,<NA>,2024-06-28 13:54:00,Aoudad,,36.5,Broadside,Level,Full Shadow,High,On_Aim_Point,Center 10,Felt_Good,,10.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.86133008, ""y"": 37.27367847, ""spatial..."
6,7,93e562ec-b88d-46c1-815b-4bd4c2ad9b7a,2024-06-28 14:15:47.566999,emily.bartee,2024-06-28 14:15:47.566999,emily.bartee,<NA>,2024-06-28 14:12:00,Lynx,,41.0,Broadside,Level,Part Shadow,High,On_Aim_Point,Center 10,Felt_Good,,10.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.86141895, ""y"": 37.27369183, ""spatial..."
7,8,9807f93b-c28c-4e9b-978c-ce56dcddb650,2024-06-28 14:30:13.139000,emily.bartee,2024-06-28 14:30:13.139000,emily.bartee,<NA>,2024-06-28 14:21:00,Medium Deer,,33.5,Broadside,Level,Full Shadow,<NA>,"On_Aim_Point,Kick_Out",Lower 12,Felt_Good,,8.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.86139168, ""y"": 37.27369622, ""spatial..."
8,9,b7ecbe85-d78b-4eb3-bdf5-2b1d346ae430,2024-06-28 14:35:44.712000,emily.bartee,2024-06-28 14:35:44.712000,emily.bartee,<NA>,2024-06-28 14:35:00,Coyote,,46.3,Broadside,Level,Full Shadow,Low,Left,Center 10,Bad_Release,,8.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.86122825, ""y"": 37.27372312, ""spatial..."
9,10,ec1aeb93-7ee4-4af1-b1e2-d0158746b925,2024-06-28 14:47:43.716000,emily.bartee,2024-06-28 14:47:43.716000,emily.bartee,<NA>,2024-06-28 14:40:00,Impala,,39.0,Broadside,Level,Full Shadow,High,On_Aim_Point,Lower 12,Felt_Good,,12.0,Emily,<NA>,<NA>,<NA>,<NA>,"{""x"": -88.86113498, ""y"": 37.27374889, ""spatial..."


Shoot Details: Index(['objectid', 'globalid', 'CreationDate', 'Creator', 'EditDate', 'Editor',
       'shoot_name', 'date_and_time_of_shoot', 'location', 'temperature',
       'shoes', 'weather_conditions_present', 'personal_conditions', 'SHAPE'],
      dtype='object')


,objectid,globalid,CreationDate,Creator,EditDate,Editor,shoot_name,date_and_time_of_shoot,location,temperature,shoes,weather_conditions_present,personal_conditions,SHAPE
0,1,7898f285-2ea8-430e-bf46-8e2631d1608f,2024-06-28 12:42:15.591000,emily.bartee,2024-06-28 12:42:15.591000,emily.bartee,Metropolis ASA Warmup Range,2024-06-28 12:41:00,Outdoor,95.0,Minimal Shoes,"Sunny,Humid",Tired,"{""x"": -88.85118948, ""y"": 37.27182899, ""spatial..."
1,2,c2bbd987-9d16-4aac-a7fd-1af6122b57be,2024-06-29 12:08:24.059000,emily.bartee,2024-06-29 12:08:24.059000,emily.bartee,Metropolis ASA Day 2,2024-06-29 12:07:00,Outdoor,95.0,Minimal Shoes,"Sunny,Humid","Well_Rested,Full","{""x"": -88.85091227, ""y"": 37.27516874, ""spatial..."
2,3,af40ee5c-b54f-47e2-8481-ade008fc90b1,2024-07-06 12:43:00.898999,emily.bartee,2024-07-06 12:43:00.898999,emily.bartee,Chickasaw,2024-07-06 12:42:00,Outdoor,80.0,Minimal Shoes,Sunny,Tired,"{""x"": -85.7559563, ""y"": 37.99112262, ""spatialR..."
3,4,08343d5e-d7c1-442d-98cc-654bea5acdb8,2024-07-30 05:05:24.664000,emily.bartee,2024-08-06 08:02:01.300000,emily.bartee,In Range Friday Night,2024-07-05 23:00:00,Indoor,72.0,Gym Shoes,<NA>,Tired,"{""x"": -84.26486359, ""y"": 37.59060144, ""spatial..."


### Combine and clean dataframes using SQL and Pandas

1. Populate fields<br>
    - Score to Scored Ring<br>

In [69]:
# A few changes were made to the schema mid data collection - let's clean up some of the columns.  

# For the Shot Report Out dataframe, lets cleanup the score column and the adjustment made column
# Set column 'scored_ring' equal to column 'score' where column 'scored_ring' is NaN.  Due to column differences, we need to set the column type as well.
df2_Shot['scored_ring'] = df2_Shot['scored_ring'].astype(float)
df2_Shot['score'] = df2_Shot['score'].astype(float)
df2_Shot['scored_ring'] = df2_Shot['scored_ring'].fillna(df2_Shot['score'])
#display(df2_Shot)

2. Lets combine the three separate dataframes into a single dataframe using the merge function from pandas. This will be done based on common fields and data values between the dataframes.

In [70]:
#  Here are our three dataframes, for reference:  df1_Animal, df2_Shot, df3_Shoot

#  First, we will do a merge on df1_Animal to df2_Shot (df1_Animal.TargetName = df2_Shot.target).  Then, we will do another merge between the new dataframe and the df3_Shoot data frame (new.date_and_time = df3_Shoot.date_and_time_of_Shoot) based on the date portion of the datetime fields. 

# Merge df1_Animal and df2_Shot on 'TargetName' and 'target'
merged_df = pd.merge(df2_Shot, df1_Animal,  left_on='target', right_on='Target Name', how='left')
#display(merged_df)


# Extract the date portion from 'date_and_time' and 'date_and_time_of_Shoot'
merged_df['date'] = pd.to_datetime(merged_df['date_and_time']).dt.date
df3_Shoot['date'] = pd.to_datetime(df3_Shoot['date_and_time_of_shoot']).dt.date

# Merge the resulting DataFrame with df3_Shoot on the date portion of the datetime strings
final_merged_df = pd.merge(merged_df, df3_Shoot, on='date', how='inner')

# # Drop the extra 'date' column
final_merged_df.drop(columns=['date'], inplace=True)
#display(final_merged_df)

3. Clean data frames using pandas
<p>The survey123 mobile applications used to collect information on the overall shoot details and shot report out used functionality called "Multi-Select" fields.  This meant the archer was able to select multiple values for a single question if the values were all relevant, streamlining the data entry process. For example, when recording the position of the animal, the archer could select if it was Quartered_to, Quartered_From, Broadside, Leaning_Forward, or Leaning_Back all from the same question.  </p>

<p>When multiple answers are selected, the survey concatenates the answers into a string in a single field.  In order to perform analysis on the recorded values for the multi-select fields, we need to break out and add unique columns to our data frames.  To do this, we'll create new columns in our merged dataframe and use string operations to populate the columns based on the values present in the concatenated fields.</p>

Create flag fields on multi-select questions
-  Adjustment (adjustment)
    -  Increase_Yardage, Decrease_Yardage, Sight_Left, Sight_Right
-  Position (position)
    -  Quartered_to, Quartered_From, Broadside, Leaning_Forward, Leaning_Back
-  Shot Placement (shot_placement)
    -  On_Aim_Point, High, Low, Left, Right, Miss, Kick_Out
-  Shot Execution (shot_execution)
    -  Unsure, Felt_Good, Bad_Shoulder, Bad_Release, Movement_in_Shot, Unable_to_Center_Bubble, Dip, Push_Up, Glare, other
-  Release Details (release_details)
    -  Too_Soon, Fast, Slow, Off_Face, Hand_Moved
-  Weather Conditions Present (weather_conditions_present)
    -  Sunny, Cloudy, Humid, Windy, Foggy, Light_Rain, Heavy_Rain, Stormy
-  Personal Conditions (personal_conditions)
    -  Well_Rested, Tired, Sick, Sore, Hungry, Full

In [71]:
# Define the values to check
adjustment = ['Increase_Yardage','Decrease_Yardage','Sight_Left','Sight_Right']
position = ['Quartered_to','Quartered_From','Broadside','Leaning_Forward','Leaning_Back']
shot_placement = ['On_Aim_Point','High','Low','Left','Right','Miss, Kick_Out']
shot_execution = ['Unsure','Felt_Good','Bad_Shoulder','Bad_Release','Movement_in_Shot','Unable_to_Center_Bubble','Dip','Push_Up','Glare','other']
release_details = ['Too_Soon','Fast','Slow','Off_Face','Hand_Moved']
weather_conditions_present = ['Sunny','Cloudy','Humid','Windy','Foggy','Light_Rain','Heavy_Rain','Stormy']
personal_conditions = ['Well_Rested','Tired','Sick','Sore','Hungry','Full']

# Function to create new columns based on values in a specific column
def create_columns_based_on_values(df, column_name, values_list):
    if column_name in df.columns:
        for value in values_list:
            #df[value] = df[column_name].apply(lambda x: 1 if value in x else 0)
            df[value] = df[column_name].fillna('').apply(lambda x: 1 if value in x else 0)
    else:
        print(f"Column '{column_name}' does not exist in the DataFrame.")

# Create new columns for each category
create_columns_based_on_values(final_merged_df, 'adjustment', adjustment)
create_columns_based_on_values(final_merged_df, 'shot_execution', shot_execution)
create_columns_based_on_values(final_merged_df, 'position', position)
create_columns_based_on_values(final_merged_df, 'release_details', release_details)
create_columns_based_on_values(final_merged_df, 'shot_placement', shot_placement)
create_columns_based_on_values(final_merged_df, 'weather_conditions_present', weather_conditions_present)
create_columns_based_on_values(final_merged_df, 'personal_conditions', personal_conditions)


display(final_merged_df)

,objectid_x,globalid_x,CreationDate_x,Creator_x,EditDate_x,Editor_x,target_order,date_and_time,target,untitled_question_3_other,range,position,elevation,lighting,shot_confidence,shot_placement,aim_placement,shot_execution,shot_execution_other,score,shooter_name,release_details,scored_ring,adjustment_made,adjustment,SHAPE_x,OBJECTID,Target Name,Target Color,Target Size,Target Pattern,objectid_y,globalid_y,CreationDate_y,Creator_y,EditDate_y,Editor_y,shoot_name,date_and_time_of_shoot,location,temperature,shoes,weather_conditions_present,personal_conditions,SHAPE_y,Increase_Yardage,Decrease_Yardage,Sight_Left,Sight_Right,Unsure,Felt_Good,Bad_Shoulder,Bad_Release,Movement_in_Shot,Unable_to_Center_Bubble,Dip,Push_Up,Glare,other,Quartered_to,Quartered_From,Broadside,Leaning_Forward,Leaning_Back,Too_Soon,Fast,Slow,Off_Face,Hand_Moved,On_Aim_Point,High,Low,Left,Right,"Miss, Kick_Out",Sunny,Cloudy,Humid,Windy,Foggy,Light_Rain,Heavy_Rain,Stormy,Well_Rested,Tired,Sick,Sore,Hungry,Full
0,1,18762273-14e9-4bb4-836b-99d5a2fb6ff8,2024-06-28 13:06:53.326999,emily.bartee,2024-06-28 13:06:53.326999,emily.bartee,1.0,2024-06-28 12:42:00,Javelina,,37.6,Broadside,Level,Part Shadow,High,On_Aim_Point,Lower 12,Felt_Good,,12.0,Emily,<NA>,12.0,<NA>,<NA>,"{""x"": -88.85113242, ""y"": 37.27184107, ""spatial...",14,Javelina,Black,Small,Yes,1,7898f285-2ea8-430e-bf46-8e2631d1608f,2024-06-28 12:42:15.591000,emily.bartee,2024-06-28 12:42:15.591000,emily.bartee,Metropolis ASA Warmup Range,2024-06-28 12:41:00,Outdoor,95.0,Minimal Shoes,"Sunny,Humid",Tired,"{""x"": -88.85118948, ""y"": 37.27182899, ""spatial...",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
1,2,2ed4aa14-1f33-4dd4-bb7a-fd11613915d2,2024-06-28 13:11:29.308999,emily.bartee,2024-06-28 13:11:29.308999,emily.bartee,2.0,2024-06-28 13:07:00,Grazing Doe,,30.4,Broadside,Level,Full Shadow,Low,"High,Left",Lower 12,Bad_Release,,10.0,Emily,<NA>,10.0,<NA>,<NA>,"{""x"": -88.85128708, ""y"": 37.27157439, ""spatial...",10,Grazing Doe,Brown,Big,No,1,7898f285-2ea8-430e-bf46-8e2631d1608f,2024-06-28 12:42:15.591000,emily.bartee,2024-06-28 12:42:15.591000,emily.bartee,Metropolis ASA Warmup Range,2024-06-28 12:41:00,Outdoor,95.0,Minimal Shoes,"Sunny,Humid",Tired,"{""x"": -88.85118948, ""y"": 37.27182899, ""spatial...",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
2,3,30fe7f13-d439-4568-ba63-3b512976126c,2024-06-28 13:17:34.042000,emily.bartee,2024-06-28 13:17:34.042000,emily.bartee,3.0,2024-06-28 13:15:00,Brown Bear,,39.3,Broadside,Level,Full Shadow,Low,Left,Lower 12,"other,Felt_Good",Pull through,10.0,Emily,<NA>,10.0,<NA>,<NA>,"{""x"": -88.85100958, ""y"": 37.27168401, ""spatial...",7,Brown Bear,Brown,Medium,No,1,7898f285-2ea8-430e-bf46-8e2631d1608f,2024-06-28 12:42:15.591000,emily.bartee,2024-06-28 12:42:15.591000,emily.bartee,Metropolis ASA Warmup Range,2024-06-28 12:41:00,Outdoor,95.0,Minimal Shoes,"Sunny,Humid",Tired,"{""x"": -88.85118948, ""y"": 37.27182899, ""spatial...",0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
3,4,5c5f1906-e865-4bac-a20b-cf30e93321bb,2024-06-28 13:22:40.687000,emily.bartee,2024-06-28 13:22:40.687000,emily.bartee,4.0,2024-06-28 13:19:00,Grazing Doe,,31.3,Broadside,Level,Full Shadow,High,High,Lower 12,Push_Up,,10.0,Emily,<NA>,10.0,<NA>,<NA>,"{""x"": -88.85095463, ""y"": 37.27159442, ""spatial...",10,Grazing Doe,Brown,Big,No,1,7898f285-2ea8-430e-bf46-8e2631d1608f,2024-06-28 12:42:15.591000,emily.bartee,2024-06-28 12:42:15.591000,emily.bartee,Metropolis ASA Warmup Range,2024-06-28 12:41:00,Outdoor,95.0,Minimal Shoes,"Sunny,Humid",Tired,"{""x"": -88.85118948, ""y"": 37.27182899, ""spatial...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
4,5,0d1918ec-6ca2-42ff-bc48-fff9edf9e7d1,2024-06-28 13:29:50.051000,emily.bartee,2024-06-28 13:29:50.051000,emily.bartee,<NA>,2024-06-28 13:27:00,Black Panther,,41.0,Quartered_From,Level,Full Shado

Create graphics
- Shot placement on color
- Shot placement on aim point
- Shot Placement on range
- Shot placement on 
- Animal Size on color
- Shot placement on position
- Overall Shot Placement
- Release Details on scored ring


Create Readme
    Modules Needed
    

Future Work
    1.  Identify further parameters for performing analysis
    2.  Fine tune the shoot report mobile survey to collect additional informaiton (i.e. which direction is the target facing), populate defaults, and improve the data entry process, and allow for open ranges. 
    3.  Create a PDF or Word Report for daily shoot summaries
    
    